In [ ]:
import mnist
import torch
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

try:
    from sklearnex import patch_sklearn
    patch_sklearn()
except:
    print("No scikit-learn-intelex found.  We go on with the classic implementation.")

import time
from pathlib import Path

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformaciones para normalizar los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Cargar los datos de entrenamiento y prueba
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# Crear dataloaders para iterar sobre los datos en lotes
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Capas de la red
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

net = Net()
print(net)

import torch.optim as optim

# Crear una instancia del modelo
model = Net()

# Definir una función de pérdida
criterion = nn.NLLLoss()

# Definir un optimizador
optimizer = optim.SGD(model.parameters(), lr=0.003)

def evaluate(model, data_loader):
    correct = 0
    true_positives = 0
    false_negatives = 0

    with torch.no_grad():
        for images, labels in data_loader:
            # Get predictions from the model
            output = model(images)
            _, predicted = torch.max(output.data, 1)

            # Count correct predictions
            correct += (predicted == labels).sum().item()

            # Count true positives and false negatives
            for i in range(len(labels)):
                if predicted[i] == labels[i] == 1:
                    true_positives += 1
                elif predicted[i] == 0 and labels[i] == 1:
                    false_negatives += 1
    
    # Calculate accuracy and recall
    accuracy = correct / len(data_loader.dataset)
    recall = true_positives / (true_positives + false_negatives)
    
    return accuracy, recall
# Entrenar la red neuronal
epochs = 100
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        # Limpiar los gradientes
        optimizer.zero_grad()
        
        # Calcular la salida del modelo
        output = model(images)
        
        # Calcular la pérdida
        loss = criterion(output, labels)
        
        # Calcular los gradientes
        loss.backward()
        
        # Actualizar los pesos
        optimizer.step()
        
        running_loss += loss.item()
    else:
        # Evaluate the model on the training and test sets
        train_acc, train_recall = evaluate(model, train_loader)
        test_acc, test_recall = evaluate(model, test_loader)

        print(f"Epoch: {e+1}/{epochs}.. Training loss: {running_loss/len(train_loader)}.."
              f" Train accuracy: {train_acc}, Train recall: {train_recall}.."
              f" Test accuracy: {test_acc}, Test recall: {test_recall}")
        



#########Prueba criterios
#def evaluatef(model, data_loader):
#    correct = 0
#     total = 0
#     true_positives = 0
#     actual_positives = 0
#     predicted_positives = 0
#     for images, labels in data_loader:
#         output = model(images)
#         _, predicted = torch.max(output.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         true_positives += ((predicted == 1) & (labels == 1)).sum().item()
#         actual_positives += (labels == 1).sum().item()
#         predicted_positives += (predicted == 1).sum().item()
# 
#     accuracy = 100 * correct / total
#     recall = 100 * true_positives / actual_positives

#     print(f"Accuracy: {accuracy:.2f}%")
#     print(f"Recall: {recall:.2f}%")
#     print("-" * 40) # Add a separator line

# evaluatef(net, test_loader)
####################
# Guardar el estado del modelo
#torch.save(model.state_dict(), 'model.pth')

###########################################################

# Crear una nueva instancia del modelo
#model = Net()

# Cargar el estado del modelo
#model.load_state_dict(torch.load('model.pth'))

# Evaluar el modelo
#model.eval()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)
